In [19]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
import functions
import scipy
import math
import operator
from functions import *
import itertools
from sklearn.ensemble import StackingClassifier

In [20]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [21]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() #StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [24]:
#import models with pickle
import pickle
clf1 = pickle.load(open('models/catboost_model.sav', 'rb'))
clf2 = pickle.load(open('models/extra_model.sav', 'rb'))
clf3 = pickle.load(open('models/svm_model.sav', 'rb'))
clf4 = pickle.load(open('models/xgboost_model.sav', 'rb'))
clf5 = pickle.load(open('models/random_forest_model.sav', 'rb'))

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
final_estimator = SVC(probability = True)
eclf2 = StackingClassifier(estimators=[('svm', clf3), ('xgboost', clf4), ('rf',clf5)], final_estimator=final_estimator)
#scores = cross_val_score(eclf2, x_train, np.ravel(y_train), cv=5, scoring='f1_micro' , n_jobs=-1, verbose=2)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
final_estimator = SVC(probability = True)
eclf2 = StackingClassifier(estimators=[ ('catboost',clf1), ('extra', clf2), ('svm', clf3), ('xgboost', clf4), ('rf',clf5)], final_estimator=final_estimator)
#scores = cross_val_score(eclf2, x_train, np.ravel(y_train), cv=5, scoring='f1_micro' , n_jobs=8, verbose=2)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:    3.5s remaining:    5.3s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:    3.7s finished


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\sklearn\ensemble\_stacking.py", line 658, in fit
    return super().fit(X, y_encoded, sample_weight)
  File "C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\sklearn\ensemble\_stacking.py", lin

In [6]:
print(scores.mean())
print(scores.std())
#0.7148702880620725 / 0.01066740914333765
#0.7287474034701859 / 0.008069266211889234
#0.7346098332111438 / 0.011019933020849262 578 features with logistic regression
#0.7379316730816228 / 0.012364254009713383 578 features with svm
#0.816495028409091 / 0.0054742345511349854 1000 features

nan
nan


In [32]:
%%time
eclf2.fit(x_train, np.ravel(y_train))
y_test = eclf2.predict(x_test)
data_y_test = {
    'id': [i for i in range(len(y_test))],
    'y': y_test
}
df_y_test = pd.DataFrame(data_y_test)
df_y_test.to_csv('outputs/output_ensemble_stacking.csv', index=False)

CPU times: total: 1h 11min 33s
Wall time: 34min 40s


In [33]:
import joblib
joblib.dump(eclf2, 'models/eclf2.sav')

['models/eclf2.sav']